# retriever 형태 확인

In [2]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect("fridges.db")
df = pd.read_sql("SELECT * FROM menu", conn)

df['page_content'] = df['name'] + " " + df['ingredients'] + " " + df['recipe']
df.drop(columns=['name', 'ingredients', 'recipes'], inplace=True)
conn.close()
df.columns

Index(['id', 'img', 'video', 'ingredients_list', 'page_content'], dtype='object')

In [1]:
import sqlite3
import pandas as pd

from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever


from dotenv import load_dotenv

load_dotenv()

True

In [2]:
def load(case):
    if case == "funs":
        conn = sqlite3.connect("funs.db")
        df = pd.read_sql("SELECT * FROM menu", conn)

        df['page_content'] = df['name'] + " ||| " + df['ingredients'] + " ||| " + df['recipe']
        df.drop(columns=['name', 'ingredients', 'recipe'], inplace=True)
        conn.close()
        loader = DataFrameLoader(df, page_content_column="page_content")
        docs = loader.load()
        return docs
    
    elif case == "ref":
        conn = sqlite3.connect("fridges.db")
        df = pd.read_sql("SELECT * FROM menu", conn)

        df['page_content'] = df['name'] + " ||| " + df['ingredients'] + " ||| " + df['recipe']
        df.drop(columns=['name', 'ingredients', 'recipe'], inplace=True)
        conn.close()
        loader = DataFrameLoader(df, page_content_column="page_content")
        docs = loader.load()
        return docs
    
    else:
        conn = sqlite3.connect("man.db")
        df = pd.read_sql("SELECT * FROM processed_data", conn)

        df['page_content'] = df['name'] + " ||| " + df['ingredients'] + " ||| " + df['recipe'] + " ||| " + df['category'] + " ||| " + df['info'] + " ||| " + df['intro']
        df.drop(columns=['name', 'ingredients', 'recipe', "info", "intro"], inplace=True)
        conn.close()
        loader = DataFrameLoader(df, page_content_column="page_content")
        splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(model_name='gpt-4o-mini', chunk_size=1000, chunk_overlap=0)
        docs = loader.load_and_split(splitter)
        return docs

In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# docs = load("funs")
# fais = FAISS.from_documents(docs, embedding=embeddings)
# fais.save_local("fun_faiss")

In [ ]:
# fais = FAISS.load_local("fun_faiss", embeddings, allow_dangerous_deserialization=True)
# fais.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.05}).invoke("사과")

c:\Users\Playdata\SKN06\last_prj\venv\Lib\site-packages\langchain_core\vectorstores\base.py:1076: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='e94ba6fc-e82b-4cea-9433-0eb33c5b52a1', metadata={'id': 1691, 'img': 'https://pbbsres.kbs.co.kr/t2019-0286-04-296340/2024/08/23/2729082_5.png', 'video': 'https://www.youtube.com/watch?v=iFqp86-Z3W0&pp=ygU07KeE7ISc7JewIC0gJ-yCrOqzvOuyoOydtOq4gCcg66CI7Iuc7ZS8IO2OuOyKpO2GoOuekQ%3D%3D'}, page_content="진서연 - '사과베이글' 레시피 ||| 사과 1개, 땅콩버터, 그래놀라, 그릭요구르트 ||| 1. 사과는 2cm 두께로 사과 꼭지와 수직 방향이 되게 자른다. 2. 사과의 정가운데 씨가 있는 부분은 칼로 썰어 제거한다. 3. 사과에 땅콩버터를 바르고 그래놀라, 그릭요구르트, 사과 순으로 쌓으면 완성"), np.float32(-0.056272507)), (Document(id='2c3a0fa6-c4af-4b7c-bcdf-cb8d27581beb', metadata={'id': 1778, 'img': 'https://pbbsres.kbs.co.kr/t2019-0286-04-296340/2024/12/27/2835445_5.jpg', 'video': 'https://www.youtube.com/watch?v=90JKu3kWSGY&pp=ygU17J6l66-87Zi4IC0gJ-yCrOqzvCDsvIDsnbTtgawnIOugiOyLnO2UvCDtjrjsiqTthqDrnpE%3D'}, page_content="장민호 - '사과 케이크' 레시피 |||

[]

In [ ]:
# docs = load("ref")
# fais = FAISS.from_documents(docs, embedding=embeddings)
# fais.save_local("ref_faiss")

In [ ]:
# docs = load("man")
# fais = FAISS.from_documents(docs, embedding=embeddings)
# fais.save_local("man_faiss")

In [3]:
def load_retriever(case, faiss_path):
    """ retriever 로드 함수"""
    docs = load(case)

    # vectordb 로드
    fais = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)

    # BM25 retriever, FAISS retriever 앙상블
    bm25_retr = BM25Retriever.from_documents(docs)
    bm25_retr.k = 3
    fais_retr = fais.as_retriever(search_kwargs={"k": 3})
    
    return bm25_retr, fais_retr

In [7]:
rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

retriever = EnsembleRetriever(retrievers=[rbm25_retr, rfais_retr, fbm25_retr, ffais_retr, mbm25_retr, mfais_retr],)

In [10]:
retriever.invoke("오렌지")

[Document(metadata={'id': 47620, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2021/05/07/91f9e07b597a37e3b62d6731c0e3e7891.jpg', 'views': '791', 'video': '', 'category': '초스피드', 'date': '2021-05-07'}, page_content='오렌지 예쁘게 잘라서 먹기 ||| 오렌지 ||| 오렌지 반으로 잘라 준다. 양 끝을 조금 잘라낸다. 숟가락을 이용해 자른 오렌지 속을 파내줍니다. 양 끝을 잘라 놓은 꼭지 부분을 속파낸 오렌지에 넣어 줍니다. 오렌지 과육 먹기 좋게 썰어 오렌지 컵에 담아 주면 완성입니다. ||| 초스피드 ||| 1인분 10분 이내 아무나 ||| 오렌지 예쁘게 잘라 손님상에 담아내면 좋겠죠'),
 Document(metadata={'id': 361, 'img': 'https://fs.jtbc.co.kr/prog/enter/janggo/Img/20160629_103245_6624.jpg', 'video': 'https://tv.jtbc.co.kr/vod/pr10010331/pm10031085/vo10111615/view', 'ingredients_list': "['우유', '두유', '고구마 아이스크림', '오렌지', '바나나', '달걀', '버터', '밀가루', '설탕']"}, page_content="샘킴 '크레이지 크레페' ||| 우유,    두유,    고구마 아이스크림,    오렌지,    바나나,    달걀,    버터,    밀가루,    설탕 |||   1. 팬에 버터를 녹인다.  2. 밀가루를 체에 밭쳐 곱게 친다.  3. 핸드블랜더에 우유, 설탕, 달걀, 체 친 밀가루를 넣는다.  4. 3에 녹인 버터를 넣고 갈아 크레페 반죽을 만든다.  5. 냄비에 버터, 밀가루, 두유, 고구마 아이스크림을 넣고 졸여 고구마소스를 만든다.  6. 마른 팬에 오렌지즙을 넣는다.  7. 오

# ContextualCompressionRetriever

In [32]:
import sqlite3
import pandas as pd
from typing import List
from pydantic import BaseModel, Field

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers import EnsembleRetriever, MergerRetriever, ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from langchain_openai import OpenAI
from langchain_core.messages import BaseMessage
from langchain_core.chat_history import BaseChatMessageHistory

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
## 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

## 모델
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
def load(case):
    if case == "funs":
        conn = sqlite3.connect("funs.db")
        df = pd.read_sql("SELECT * FROM menu", conn)

        df['page_content'] = df['name'] + " ||| " + df['ingredients'] + " ||| " + df['recipe']
        df.drop(columns=['name', 'ingredients', 'recipe'], inplace=True)
        conn.close()
        loader = DataFrameLoader(df, page_content_column="page_content")
        docs = loader.load()
        return docs
    
    elif case == "ref":
        conn = sqlite3.connect("fridges.db")
        df = pd.read_sql("SELECT * FROM menu", conn)

        df['page_content'] = df['name'] + " ||| " + df['ingredients'] + " ||| " + df['recipe']
        df.drop(columns=['name', 'ingredients', 'recipe'], inplace=True)
        conn.close()
        loader = DataFrameLoader(df, page_content_column="page_content")
        docs = loader.load()
        return docs
    
    else:
        conn = sqlite3.connect("man.db")
        df = pd.read_sql("SELECT * FROM processed_data", conn)

        df['page_content'] = df['name'] + " ||| " + df['ingredients'] + " ||| " + df['recipe'] + " ||| " + df['category'] + " ||| " + df['info'] + " ||| " + df['intro']
        df.drop(columns=['name', 'ingredients', 'recipe', "info", "intro"], inplace=True)
        conn.close()
        loader = DataFrameLoader(df, page_content_column="page_content")
        splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(model_name='gpt-4o-mini', chunk_size=1000, chunk_overlap=0)
        docs = loader.load_and_split(splitter)
        return docs

In [4]:
def load_retriever(case, faiss_path):
    """ retriever 로드 함수"""
    docs = load(case)

    # vectordb 로드
    fais = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)

    # BM25 retriever, FAISS retriever 앙상블
    bm25_retr = BM25Retriever.from_documents(docs)
    bm25_retr.k = 3
    fais_retr = fais.as_retriever(search_type="mmr", search_kwargs={"k": 3})
    
    return bm25_retr, fais_retr

In [5]:
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    """인메모리 히스토리 클래스"""
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

In [6]:
# 메모리 정의
store = {}
def get_session_history(user_id: str, history_id: str) -> BaseChatMessageHistory:
    if (user_id, history_id) not in store:
        store[(user_id, history_id)] = InMemoryHistory()
    print(f"현재 저장된 히스토리: {store}") # 디버깅용
    return store[(user_id, history_id)]

In [7]:
def version1(): # 6개 앙상블 + ContextualCompressionRetriever(CrossEncoderReranker)
    # retriever 로드
    rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
    fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
    mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

    ensembled = EnsembleRetriever(retrievers=[rbm25_retr, rfais_retr, fbm25_retr, ffais_retr, mbm25_retr, mfais_retr])

    hmodel = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=hmodel, top_n=3)

    retriever = ContextualCompressionRetriever(base_retriever=ensembled, base_compressor=compressor)
    return retriever

In [33]:
def version2(): # 6개 앙상블 + ContextualCompressionRetriever(LLMChainExtractor)
    # retriever 로드
    rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
    fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
    mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

    ensembled = EnsembleRetriever(retrievers=[rbm25_retr, rfais_retr, fbm25_retr, ffais_retr, mbm25_retr, mfais_retr])
    
    llm = OpenAI(temperature=0)
    compressor = LLMChainFilter.from_llm(llm)

    retriever = ContextualCompressionRetriever(base_retriever=ensembled, base_compressor=compressor)
    return retriever

In [9]:
def version3(): # 6개 merge + ContextualCompressionRetriever(CrossEncoderReranker)
    # retriever 로드
    rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
    fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
    mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

    merged = MergerRetriever(retrievers=[rbm25_retr, rfais_retr, fbm25_retr, ffais_retr, mbm25_retr, mfais_retr])
    hmodel = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=hmodel, top_n=3)

    retriever = ContextualCompressionRetriever(base_retriever=merged, base_compressor=compressor)
    return retriever

In [34]:
def version4(): # 6개 merge + ContextualCompressionRetriever(LLMChainExtractor)
    # retriever 로드
    rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
    fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
    mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

    merged = MergerRetriever(retrievers=[rbm25_retr, rfais_retr, fbm25_retr, ffais_retr, mbm25_retr, mfais_retr])

    llm = OpenAI(temperature=0)
    compressor = LLMChainFilter.from_llm(llm)

    retriever = ContextualCompressionRetriever(base_retriever=merged, base_compressor=compressor)
    return retriever

In [11]:
def version5(): # 개별 문서 앙상블 -> MergeRetriever + ContextualCompressionRetriever(CrossEncoderReranker)
    # retriever 로드
    rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
    fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
    mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

    ensemble1 = EnsembleRetriever(retrievers=[rbm25_retr, rfais_retr],) # weights=[0.25, 0.25, 0.25, 0.25],) # weight: retriever 별 가중치 조절 가능
    ensemble2 = EnsembleRetriever(retrievers=[fbm25_retr, ffais_retr],) # weights=[0.25, 0.25, 0.25, 0.25],) # weight: retriever 별 가중치 조절 가능
    ensemble3 = EnsembleRetriever(retrievers=[mbm25_retr, mfais_retr],) # weights=[0.25, 0.25, 0.25, 0.25],) # weight: retriever 별 가중치 조절 가능
    merged = MergerRetriever(retrievers=[ensemble1, ensemble2, ensemble3])
    
    hmodel = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
    compressor = CrossEncoderReranker(model=hmodel, top_n=3)

    retriever = ContextualCompressionRetriever(base_retriever=merged, base_compressor=compressor)
    return retriever

In [35]:
def version6(): # 개별 문서 앙상블 -> MergeRetriever + ContextualCompressionRetriever(LLMChainExtractor)
    # retriever 로드
    rbm25_retr, rfais_retr = load_retriever("ref", "ref_faiss") # 냉장고를 부탁해
    fbm25_retr, ffais_retr = load_retriever("funs", "fun_faiss") # 편스토랑
    mbm25_retr, mfais_retr = load_retriever("man", "man_faiss") # 만개의 레시피

    ensemble1 = EnsembleRetriever(retrievers=[rbm25_retr, rfais_retr],) # weights=[0.25, 0.25, 0.25, 0.25],) # weight: retriever 별 가중치 조절 가능
    ensemble2 = EnsembleRetriever(retrievers=[fbm25_retr, ffais_retr],) # weights=[0.25, 0.25, 0.25, 0.25],) # weight: retriever 별 가중치 조절 가능
    ensemble3 = EnsembleRetriever(retrievers=[mbm25_retr, mfais_retr],) # weights=[0.25, 0.25, 0.25, 0.25],) # weight: retriever 별 가중치 조절 가능
    merged = MergerRetriever(retrievers=[ensemble1, ensemble2, ensemble3])
    
    llm = OpenAI(temperature=0)
    compressor = LLMChainFilter.from_llm(llm)

    retriever = ContextualCompressionRetriever(base_retriever=merged, base_compressor=compressor)
    return retriever

In [13]:
version1().invoke("땅콩이 안 들어간 요리")

c:\Users\Playdata\SKN06\last_prj\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Playdata\SKN06\last_prj\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to acti

[Document(metadata={'id': 198926, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2016/01/19/86fecc262a430c0e27d1489129291a2d1.jpg', 'views': '12,530', 'video': '', 'category': '일상', 'date': '2016-01-17'}, page_content='실치볶음 ||| 실치, 땅콩, 마늘, [양념] 간장, 올리고당, 설탕, 맛술, 통깨 ||| 땅콩이 너무 맛이 없는 걸 사와서 줄어들지 않고 있어요. 팬에 한번 볶아서 고소함을 업시킨 후에 넣었어요. 실치가 너무 작은 양밖에 안 남아서 개량 없이 눈대중으로 넣었어요. 간장, 맛술, 올리고당, 마늘 넣고 부글부글 끓여 주세요. 실치랑 땅콩을 넣어 주세요. 실. 치도 한번 볶아서 넣어야 하는데 깜박 했다지요. 잠시 저어 주세요. 바삭하라고 설탕도 넣었구요. 통깨도 솔솔 뿌려줬어요. 접시에 담아 주면 끝. ||| 일상 ||| 2인분 30분 이내 초급 ||| 냉동실에 남아 있는 실치를 꺼내 땅콩과 함께 휘리릭 볶은 실치볶음입니다 땅콩이 들어가 더욱 고소하고 달콤해요 마땅한 밑반찬이 없을 때 간단하게 만들어 먹을 수 있는 칼슘듬뿍 실치반찬이예요'),
 Document(metadata={'id': 12929, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2020/12/12/e8c926514ccd9129c57c1d4e78b45ad81.jpg', 'views': '2,994', 'video': '', 'category': '일상', 'date': '2020-12-12'}, page_content='땅콩조림 ||| 생땅콩-500g, [양념] 물-200ml, 간장-5큰술, 물엿-2큰술, 설탕-1큰술, 통깨, 적당량 ||| 생 땅콩을 물에 한번 깨끗하게 씻어 줍니다. 냄비에 땅콩을 넣고 땅콩이 잠길 만큼 물을 부어 주고 삶아 줍니다. 땅콩을

In [36]:
version2().invoke("땅콩이 안 들어간 요리")

[Document(id='aa9af9bc-8b38-4f6d-9550-1e58d98549a7', metadata={'id': 916, 'img': '', 'video': 'https://www.youtube.com/watch?v=xJ5O8jZa2xY&pp=ygUc7J207JuQ7J28ICftlZzsi50g67iM65-w7LCcJw%3D%3D', 'ingredients_list': "['솔잎', '굴비채', '눈개승마', '아가베 시럽', '유자청', '된장', '요리술', '즉석밥', '소금', '다진 마늘', '다진 파', '고춧가루', '깨', '참기름', '양조간장']"}, page_content="이원일 '한식 브런찜' ||| 솔잎,    굴비채,    눈개승마,    아가베 시럽,    유자청,    된장,    요리술,    즉석밥,    소금,   다진 마늘,    다진 파,    고춧가루,    깨,    참기름,    양조간장 |||   1. 끓는 물에 된장을 풀고 요리술을 넣어 끓인다.  2. 찜기에 솔잎을 올리고 굴비채를 찐다.  3. 즉석밥을 전자레인지에 넣고 익힌다.  4. 끓는 물에 소금을 넣고 눈개승마를 데친다.  5. 데친 눈개승마를 차가운 물에 담가 놓는다.  6. 다진 마늘, 다진 파, 된장, 아가베 시럽을 섞는다.  7. 6에 고춧가루, 깨를 갈아 넣어 나물 양념을 만든다.  8. 물기를 뺀 눈개승마에 나물 양념과 참기름을 넣고 무친다.  9. 접시에 밥과 나물을 올린다.  10. 대파, 양조간장, 유자청, 아가베 시럽, 물을 섞어 굴비채 양념을 만든다.  11. 찐 굴비채를 접시에 담고 굴비채 양념을 올려 완성한다."),
 Document(metadata={'id': 602, 'img': 'https://pbbsres.kbs.co.kr/t2019-0286-04-296340/2021/12/24/1509632_batch_치즈퐁듀.png', 'video': 'https://www.youtube.com/watch?v=x-LsihjKh

In [15]:
version3().invoke("땅콩이 안 들어간 요리")

[Document(metadata={'id': 198926, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2016/01/19/86fecc262a430c0e27d1489129291a2d1.jpg', 'views': '12,530', 'video': '', 'category': '일상', 'date': '2016-01-17'}, page_content='실치볶음 ||| 실치, 땅콩, 마늘, [양념] 간장, 올리고당, 설탕, 맛술, 통깨 ||| 땅콩이 너무 맛이 없는 걸 사와서 줄어들지 않고 있어요. 팬에 한번 볶아서 고소함을 업시킨 후에 넣었어요. 실치가 너무 작은 양밖에 안 남아서 개량 없이 눈대중으로 넣었어요. 간장, 맛술, 올리고당, 마늘 넣고 부글부글 끓여 주세요. 실치랑 땅콩을 넣어 주세요. 실. 치도 한번 볶아서 넣어야 하는데 깜박 했다지요. 잠시 저어 주세요. 바삭하라고 설탕도 넣었구요. 통깨도 솔솔 뿌려줬어요. 접시에 담아 주면 끝. ||| 일상 ||| 2인분 30분 이내 초급 ||| 냉동실에 남아 있는 실치를 꺼내 땅콩과 함께 휘리릭 볶은 실치볶음입니다 땅콩이 들어가 더욱 고소하고 달콤해요 마땅한 밑반찬이 없을 때 간단하게 만들어 먹을 수 있는 칼슘듬뿍 실치반찬이예요'),
 Document(metadata={'id': 12929, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2020/12/12/e8c926514ccd9129c57c1d4e78b45ad81.jpg', 'views': '2,994', 'video': '', 'category': '일상', 'date': '2020-12-12'}, page_content='땅콩조림 ||| 생땅콩-500g, [양념] 물-200ml, 간장-5큰술, 물엿-2큰술, 설탕-1큰술, 통깨, 적당량 ||| 생 땅콩을 물에 한번 깨끗하게 씻어 줍니다. 냄비에 땅콩을 넣고 땅콩이 잠길 만큼 물을 부어 주고 삶아 줍니다. 땅콩을

In [37]:
version4().invoke("땅콩이 안 들어간 요리")

[Document(id='aa9af9bc-8b38-4f6d-9550-1e58d98549a7', metadata={'id': 916, 'img': '', 'video': 'https://www.youtube.com/watch?v=xJ5O8jZa2xY&pp=ygUc7J207JuQ7J28ICftlZzsi50g67iM65-w7LCcJw%3D%3D', 'ingredients_list': "['솔잎', '굴비채', '눈개승마', '아가베 시럽', '유자청', '된장', '요리술', '즉석밥', '소금', '다진 마늘', '다진 파', '고춧가루', '깨', '참기름', '양조간장']"}, page_content="이원일 '한식 브런찜' ||| 솔잎,    굴비채,    눈개승마,    아가베 시럽,    유자청,    된장,    요리술,    즉석밥,    소금,   다진 마늘,    다진 파,    고춧가루,    깨,    참기름,    양조간장 |||   1. 끓는 물에 된장을 풀고 요리술을 넣어 끓인다.  2. 찜기에 솔잎을 올리고 굴비채를 찐다.  3. 즉석밥을 전자레인지에 넣고 익힌다.  4. 끓는 물에 소금을 넣고 눈개승마를 데친다.  5. 데친 눈개승마를 차가운 물에 담가 놓는다.  6. 다진 마늘, 다진 파, 된장, 아가베 시럽을 섞는다.  7. 6에 고춧가루, 깨를 갈아 넣어 나물 양념을 만든다.  8. 물기를 뺀 눈개승마에 나물 양념과 참기름을 넣고 무친다.  9. 접시에 밥과 나물을 올린다.  10. 대파, 양조간장, 유자청, 아가베 시럽, 물을 섞어 굴비채 양념을 만든다.  11. 찐 굴비채를 접시에 담고 굴비채 양념을 올려 완성한다."),
 Document(metadata={'id': 602, 'img': 'https://pbbsres.kbs.co.kr/t2019-0286-04-296340/2021/12/24/1509632_batch_치즈퐁듀.png', 'video': 'https://www.youtube.com/watch?v=x-LsihjKh

In [17]:
version5().invoke("땅콩이 안 들어간 요리")

[Document(metadata={'id': 198926, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2016/01/19/86fecc262a430c0e27d1489129291a2d1.jpg', 'views': '12,530', 'video': '', 'category': '일상', 'date': '2016-01-17'}, page_content='실치볶음 ||| 실치, 땅콩, 마늘, [양념] 간장, 올리고당, 설탕, 맛술, 통깨 ||| 땅콩이 너무 맛이 없는 걸 사와서 줄어들지 않고 있어요. 팬에 한번 볶아서 고소함을 업시킨 후에 넣었어요. 실치가 너무 작은 양밖에 안 남아서 개량 없이 눈대중으로 넣었어요. 간장, 맛술, 올리고당, 마늘 넣고 부글부글 끓여 주세요. 실치랑 땅콩을 넣어 주세요. 실. 치도 한번 볶아서 넣어야 하는데 깜박 했다지요. 잠시 저어 주세요. 바삭하라고 설탕도 넣었구요. 통깨도 솔솔 뿌려줬어요. 접시에 담아 주면 끝. ||| 일상 ||| 2인분 30분 이내 초급 ||| 냉동실에 남아 있는 실치를 꺼내 땅콩과 함께 휘리릭 볶은 실치볶음입니다 땅콩이 들어가 더욱 고소하고 달콤해요 마땅한 밑반찬이 없을 때 간단하게 만들어 먹을 수 있는 칼슘듬뿍 실치반찬이예요'),
 Document(metadata={'id': 12929, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2020/12/12/e8c926514ccd9129c57c1d4e78b45ad81.jpg', 'views': '2,994', 'video': '', 'category': '일상', 'date': '2020-12-12'}, page_content='땅콩조림 ||| 생땅콩-500g, [양념] 물-200ml, 간장-5큰술, 물엿-2큰술, 설탕-1큰술, 통깨, 적당량 ||| 생 땅콩을 물에 한번 깨끗하게 씻어 줍니다. 냄비에 땅콩을 넣고 땅콩이 잠길 만큼 물을 부어 주고 삶아 줍니다. 땅콩을

In [38]:
version6().invoke("땅콩이 안 들어간 요리")

[Document(metadata={'id': 602, 'img': 'https://pbbsres.kbs.co.kr/t2019-0286-04-296340/2021/12/24/1509632_batch_치즈퐁듀.png', 'video': 'https://www.youtube.com/watch?v=x-LsihjKhMo&pp=ygUy66WY7IiY7JiBIC0gJ-y5mOymiCDtkIHrkqQnIOugiOyLnO2UvCDtjrjsiqTthqDrnpE%3D'}, page_content="류수영 - '치즈 퐁뒤' 레시피 ||| 브로콜리 1개, 양송이 4개, 감자 3개, 소시지 4개, 빵 1개, 생마늘 2개, 화이트와인 100ml, 감자전분 1T 그뤼에르치즈 200g, 에멘탈치즈 100g ||| 찍어 먹을 재료준비 1. 브로콜리 1개를 깨끗이 씻은 후 팔팔 끓는 물에 거꾸로 넣어 돌려가며 데쳐준다. 2. 끓는 물에 양송이 4개를 넣고 3분간 데쳐준다. 3. 끓는 물에 감자 3개를 넣고 20분간 삶아준다. 4. 끓는 물에 소시지 4개를 넣고 5분간 삶아준다. 5. 데쳐준 브로콜리, 양송이, 감자, 소시지를 한 입 크기로 잘라 준비한다. 6. 빵도 한입크기에 맞게 잘라준다. 한 냄비에 양송이, 감자, 소시지 데쳐서 준비하시면 됩니다 빵은 버터가 안 들어간 식사빵이 잘 어울립니다 1. 불을 켜지 않은 상태에서 뚝배기 안을 생마늘로 문질러준다. 뚝배기 안을 마늘로 문질러 향을 입혀주는 것이 좋습니다 2. 1의 문지른 마늘 2개를 으깨서 뚝배기에 넣어준다. 3. 화이트와인 100ml와, 감자전분 1T를 넣고 잘 풀어준다. 와인과 치즈의 유화를 위해 넣어주는 것이 좋습니다 뜨거운 물에 풀게 되면 굳어버리기 때문에 불을 켜지 않고 하는 것이 좋습니다 4. 전분이 다 풀어지면 불을 켜고 그뤼에르치즈 200g, 에멘탈치즈 100g을 강판에 갈아 넣어준다. 치즈를 통으로 넣으면 잘 안 익기 때문에 가위나 강판에 갈아 넣어주는 것이 좋습니다 치즈의 비율은 취향에 따라 조절해주셔도 됩니다 꼬

In [19]:
version1().invoke("프라이팬 안 쓰는 요리")

[Document(metadata={'id': 52988, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2018/08/06/2ed3ce81dee8d119fe84bc8b7a8f95bc1.jpg', 'views': '2,520', 'video': '', 'category': '초스피드', 'date': '2018-08-06'}, page_content='불 안 쓰는 요리 맛집을 집에서 간단히 꼬막 비빔밥 ||| 꼬막 통조림-2캔, 밥-2공기, 대파 흰 부분-1개, 청양고추-3개, 양파-1/2개, 고춧가루-1큰술, 꼬막 통조림 국물-2큰술, 간장-2큰술, 참기름-1큰술, 다진 마늘-1큰술, 통깨, 약간 ||| 통조림 꼬막을 체에 밭쳐 물기를 제거해 주세요. 채소를 썰어 준비해 주세요. 양념장을 만들어 주세요. 양념한 꼬막 1/2과 밥 2공기를 넣어 잘 비벼 주세요. ||| 초스피드 ||| 2인분 10분 이내 아무나 ||| 통조림 캔으로 맛집을 따라해 봤어요'),
 Document(metadata={'id': 45373, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2023/07/05/a40563370f802b16661c25e18f397cbd1.jpg', 'views': '968', 'video': '', 'category': '초스피드', 'date': '2023-07-05'}, page_content='간단하게 만드는 시원한 오이냉국 레시피 불 안 쓰는 요리 ||| 오이-1개, 홍고추-1개, 청고추-1개, [오이냉국 밑간] 소금-0. 5T, 설탕-1T, 다진마늘-0. 5T, 연두-1T, 식초-1. 5T ||| 오이는 깨끗이 씻어 먹기 좋게 채 썰어 주세요 오이의 양끝은 쓴맛이 날 수 있으니 잘라내 주세요. 약 0. 5cm 정도로 어슷 썰고. 비슷한 굵기로 채 썰었어요. 오이냉국 밑간을 해 주세요. 먹기 전 밑간해 둔 오이냉국에 얼음을 넣고. 재료가 충분히 잠길 만큼 물

In [39]:
version2().invoke("프라이팬 안 쓰는 요리")

[Document(metadata={'id': 395, 'img': 'https://fs.jtbc.co.kr/prog/enter/janggo/Img/20151229_102856_7914.jpg', 'video': 'https://tv.jtbc.co.kr/vod/pr10010331/pm10031085/vo10089513/view', 'ingredients_list': "['브로콜리', '북어채', '새우', '다진 청양고추', '대파', '마늘', '라면', '올리브 오일']"}, page_content="오세득 '개천에서 용 났새우' ||| 브로콜리,    북어채,    새우,    다진 청양고추,    대파,    마늘,    라면,    올리브 오일 |||   1. 브로콜리를 적당한 크기로 썰어 소금을 넣은 끓는 물에 데친다.  2. 데친 브로콜리를 그릴팬에 굽는다.  3. 북어채에 기름을 두른 뒤, 전자레인지에 돌려 바삭하게 만든다.  4. 끓는 물에 라면을 삶는다.  5. 팬에 올리브 오일을 넉넉히 두른다.  6. 올리브 오일을 두른 팬에 두껍게 썬 마늘과 파를 넣고 볶아 향을 낸다.  7. 6에 새우와 다진 청양고추를 넣고 볶는다.  8. 양파를 잘게 다져 팬에 넣고 함께 볶는다.  9. 그릇에 구운 브로콜리를 담는다.  10. 삶은 면을 체에 밭쳐 물기를 뺀다.  11. 면에 재료들의 향이 밴 기름을 넣고 버무린다.  12. 면을 그릇에 담는다.  13. 면 위에 볶은 재료들을 올린다.  14. 바삭해진 북어채를 핸드블렌더에 넣고 간다.  15. 곱게 간 북어채를 요리 위에 뿌려 완성한다."),
 Document(id='cb572e30-ac11-4ee6-b8ed-1201c44c6ed1', metadata={'id': 840, 'img': 'https://fs.jtbc.co.kr/prog/enter/janggo/Img/20160412_132025_2277.jpg', 'video': 'https://tv.jtbc.co.kr/vod/pr10010331/p

In [21]:
version3().invoke("프라이팬 안 쓰는 요리")

[Document(metadata={'id': 52988, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2018/08/06/2ed3ce81dee8d119fe84bc8b7a8f95bc1.jpg', 'views': '2,520', 'video': '', 'category': '초스피드', 'date': '2018-08-06'}, page_content='불 안 쓰는 요리 맛집을 집에서 간단히 꼬막 비빔밥 ||| 꼬막 통조림-2캔, 밥-2공기, 대파 흰 부분-1개, 청양고추-3개, 양파-1/2개, 고춧가루-1큰술, 꼬막 통조림 국물-2큰술, 간장-2큰술, 참기름-1큰술, 다진 마늘-1큰술, 통깨, 약간 ||| 통조림 꼬막을 체에 밭쳐 물기를 제거해 주세요. 채소를 썰어 준비해 주세요. 양념장을 만들어 주세요. 양념한 꼬막 1/2과 밥 2공기를 넣어 잘 비벼 주세요. ||| 초스피드 ||| 2인분 10분 이내 아무나 ||| 통조림 캔으로 맛집을 따라해 봤어요'),
 Document(metadata={'id': 45373, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2023/07/05/a40563370f802b16661c25e18f397cbd1.jpg', 'views': '968', 'video': '', 'category': '초스피드', 'date': '2023-07-05'}, page_content='간단하게 만드는 시원한 오이냉국 레시피 불 안 쓰는 요리 ||| 오이-1개, 홍고추-1개, 청고추-1개, [오이냉국 밑간] 소금-0. 5T, 설탕-1T, 다진마늘-0. 5T, 연두-1T, 식초-1. 5T ||| 오이는 깨끗이 씻어 먹기 좋게 채 썰어 주세요 오이의 양끝은 쓴맛이 날 수 있으니 잘라내 주세요. 약 0. 5cm 정도로 어슷 썰고. 비슷한 굵기로 채 썰었어요. 오이냉국 밑간을 해 주세요. 먹기 전 밑간해 둔 오이냉국에 얼음을 넣고. 재료가 충분히 잠길 만큼 물

In [40]:
version4().invoke("프라이팬 안 쓰는 요리")

[Document(metadata={'id': 395, 'img': 'https://fs.jtbc.co.kr/prog/enter/janggo/Img/20151229_102856_7914.jpg', 'video': 'https://tv.jtbc.co.kr/vod/pr10010331/pm10031085/vo10089513/view', 'ingredients_list': "['브로콜리', '북어채', '새우', '다진 청양고추', '대파', '마늘', '라면', '올리브 오일']"}, page_content="오세득 '개천에서 용 났새우' ||| 브로콜리,    북어채,    새우,    다진 청양고추,    대파,    마늘,    라면,    올리브 오일 |||   1. 브로콜리를 적당한 크기로 썰어 소금을 넣은 끓는 물에 데친다.  2. 데친 브로콜리를 그릴팬에 굽는다.  3. 북어채에 기름을 두른 뒤, 전자레인지에 돌려 바삭하게 만든다.  4. 끓는 물에 라면을 삶는다.  5. 팬에 올리브 오일을 넉넉히 두른다.  6. 올리브 오일을 두른 팬에 두껍게 썬 마늘과 파를 넣고 볶아 향을 낸다.  7. 6에 새우와 다진 청양고추를 넣고 볶는다.  8. 양파를 잘게 다져 팬에 넣고 함께 볶는다.  9. 그릇에 구운 브로콜리를 담는다.  10. 삶은 면을 체에 밭쳐 물기를 뺀다.  11. 면에 재료들의 향이 밴 기름을 넣고 버무린다.  12. 면을 그릇에 담는다.  13. 면 위에 볶은 재료들을 올린다.  14. 바삭해진 북어채를 핸드블렌더에 넣고 간다.  15. 곱게 간 북어채를 요리 위에 뿌려 완성한다."),
 Document(id='cb572e30-ac11-4ee6-b8ed-1201c44c6ed1', metadata={'id': 840, 'img': 'https://fs.jtbc.co.kr/prog/enter/janggo/Img/20160412_132025_2277.jpg', 'video': 'https://tv.jtbc.co.kr/vod/pr10010331/p

In [23]:
version5().invoke("프라이팬 안 쓰는 요리")

[Document(metadata={'id': 52988, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2018/08/06/2ed3ce81dee8d119fe84bc8b7a8f95bc1.jpg', 'views': '2,520', 'video': '', 'category': '초스피드', 'date': '2018-08-06'}, page_content='불 안 쓰는 요리 맛집을 집에서 간단히 꼬막 비빔밥 ||| 꼬막 통조림-2캔, 밥-2공기, 대파 흰 부분-1개, 청양고추-3개, 양파-1/2개, 고춧가루-1큰술, 꼬막 통조림 국물-2큰술, 간장-2큰술, 참기름-1큰술, 다진 마늘-1큰술, 통깨, 약간 ||| 통조림 꼬막을 체에 밭쳐 물기를 제거해 주세요. 채소를 썰어 준비해 주세요. 양념장을 만들어 주세요. 양념한 꼬막 1/2과 밥 2공기를 넣어 잘 비벼 주세요. ||| 초스피드 ||| 2인분 10분 이내 아무나 ||| 통조림 캔으로 맛집을 따라해 봤어요'),
 Document(metadata={'id': 45373, 'img': 'https://recipe1.ezmember.co.kr/cache/recipe/2023/07/05/a40563370f802b16661c25e18f397cbd1.jpg', 'views': '968', 'video': '', 'category': '초스피드', 'date': '2023-07-05'}, page_content='간단하게 만드는 시원한 오이냉국 레시피 불 안 쓰는 요리 ||| 오이-1개, 홍고추-1개, 청고추-1개, [오이냉국 밑간] 소금-0. 5T, 설탕-1T, 다진마늘-0. 5T, 연두-1T, 식초-1. 5T ||| 오이는 깨끗이 씻어 먹기 좋게 채 썰어 주세요 오이의 양끝은 쓴맛이 날 수 있으니 잘라내 주세요. 약 0. 5cm 정도로 어슷 썰고. 비슷한 굵기로 채 썰었어요. 오이냉국 밑간을 해 주세요. 먹기 전 밑간해 둔 오이냉국에 얼음을 넣고. 재료가 충분히 잠길 만큼 물

In [41]:
version6().invoke("프라이팬 안 쓰는 요리")

[Document(metadata={'id': 395, 'img': 'https://fs.jtbc.co.kr/prog/enter/janggo/Img/20151229_102856_7914.jpg', 'video': 'https://tv.jtbc.co.kr/vod/pr10010331/pm10031085/vo10089513/view', 'ingredients_list': "['브로콜리', '북어채', '새우', '다진 청양고추', '대파', '마늘', '라면', '올리브 오일']"}, page_content="오세득 '개천에서 용 났새우' ||| 브로콜리,    북어채,    새우,    다진 청양고추,    대파,    마늘,    라면,    올리브 오일 |||   1. 브로콜리를 적당한 크기로 썰어 소금을 넣은 끓는 물에 데친다.  2. 데친 브로콜리를 그릴팬에 굽는다.  3. 북어채에 기름을 두른 뒤, 전자레인지에 돌려 바삭하게 만든다.  4. 끓는 물에 라면을 삶는다.  5. 팬에 올리브 오일을 넉넉히 두른다.  6. 올리브 오일을 두른 팬에 두껍게 썬 마늘과 파를 넣고 볶아 향을 낸다.  7. 6에 새우와 다진 청양고추를 넣고 볶는다.  8. 양파를 잘게 다져 팬에 넣고 함께 볶는다.  9. 그릇에 구운 브로콜리를 담는다.  10. 삶은 면을 체에 밭쳐 물기를 뺀다.  11. 면에 재료들의 향이 밴 기름을 넣고 버무린다.  12. 면을 그릇에 담는다.  13. 면 위에 볶은 재료들을 올린다.  14. 바삭해진 북어채를 핸드블렌더에 넣고 간다.  15. 곱게 간 북어채를 요리 위에 뿌려 완성한다."),
 Document(metadata={'id': 1067, 'img': 'https://pbbsres.kbs.co.kr/t2019-0286-04-296340/2023/03/17/2188293_batch_솔방울향스테이크.jpg', 'video': ''}, page_content="차예련 - '솔방울 훈연 스테이크' 레시피 ||| 솔방울 1개, 럼주 약간, 채끝살

# retriever + llm 비교

In [26]:
import random
import string
from textwrap import dedent

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [27]:
def mkch(retriever):
        # Prompt Template 생성
    messages = [
            ("system", dedent("""
            # instruction
            너는 사용자의 질문(question)에 맞는 요리를 알려주는 ai야.

            사용자에게 요리를 알려줄 때 요리는 context 항목에 있는 요리 중에서 알려줘야해.
            다음 조건을 참고해서 요리를 알려주면 돼.
            1. 사용자에게 요리를 추천할 때 요리 3가지를 추천한다. 그러나 사용자가 특정 요리의 레시피를 물어본 경우 해당하는 요리에 대한 정보를 제공한다. 
            2. 요리를 소개할 때 요리 이름을 먼저 언급한 뒤 간단한 요리 소개(한줄 분량), 재료, 사진 순으로 소개한다.
            2-1. 요리 이름에서 요리사 이름을 알 수 있다면 요리사 이름도 요리 이름과 같이 알려준다.
            2-2. 사진은 context에서 `img`에 있는 해당 요리의 사진 링크를 알려준다. 만약 `img`에 사진 링크가 없다면 "제공할 수 있는 사진이 없습니다."라고 답한다.
            2-3. 사진 링크를 임의로 생성하거나 다른 요리의 사진을 절대 알려주지 않는다.
            
            3. 요리 추천 시 정렬 기준은 다음과 같다.
            3-1. 사용자가 재료를 입력했을 경우, 해당 재료는 많은 순으로 먼저 정렬하고, 우선 순위가 같은 요리에 대해서는 부가적인 재료가 적은 순으로 정렬한다.
            
            4. 사용자가 요리를 고르면 레시피와 영상을 알려준다. 이때 레시피는 요약하지말고, 있는 그대로 순서대로 알려줘야한다.
            4-1. 영상은 context의 `video`에 저장된 링크 주소를 알려준다. `video`에 영상 링크가 없으면 "제공할 수 있는 영상이 없습니다." 이라고 답해야한다.
            4-2. 영상 링크 임의로 생성하거나 다른 요리의 영상 링크를 절대 알려주지 않는다.
            
            `사용자의 질문의 답을 context에서 적절한 요리를 찾지 못하면 필요에 따라 추가 정보를 사용자로부터 더 수집한 뒤 답변하고, 그럼에도 답변할 내용을 context에서 찾을 수 없으면 답변을 생성하지 말고 모른다고 대답해`
            
            # context
            
    {context}""")),
            MessagesPlaceholder(variable_name="history", optional=True),
            ("human", "{question}"),
        ]
    prompt_template = ChatPromptTemplate(messages)

    # chatting Chain 구성 retriever(관련 문서 조회) -> prompt_template(prompt 생성) model(정답) -> output parser
    chatting = {"context": itemgetter("question") | retriever, "question": itemgetter("question"), "history": itemgetter("history")} | prompt_template | model | StrOutputParser()

    chain = RunnableWithMessageHistory(
        chatting, get_session_history=get_session_history, input_messages_key="question", history_messages_key="history",
        history_factory_config=[
            ConfigurableFieldSpec(id="user_id", annotation=str, name="User ID", description="사용자 id(Unique)", default="", is_shared=True),
            ConfigurableFieldSpec(id="history_id", annotation=str, name="History ID", description="대화 기록 id(Unique)", default="", is_shared=True),
        ]
    )
    return chain

In [28]:
def mkhisid(user_id):
    """history_id 생성 함수"""
    while True:
        history_id = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(6))
        if user_id + history_id not in store.keys():
            # user_id와 history_id가 없는 경우 종료
            return history_id

In [29]:
history_id = mkhisid("user_id")
chain = mkch(version5())
while True:
    query = input("메시지 입력 > ")
    if query == "종료":
        break
    res = chain.invoke({"question": query}, config={"configurable": {"user_id": "user_id", "history_id": history_id}})
    print(res)

현재 저장된 히스토리: {('user_id', 'GKEC65'): InMemoryHistory(messages=[])}
땅콩 알레르기가 있으시다면 땅콩을 사용하지 않는 쿠키 레시피를 추천해 드리겠습니다. 다음과 같은 쿠키를 고려해 보세요:

1. **버터 쿠키**
   - 고소한 맛과 바삭한 식감이 매력적인 기본적인 버터 쿠키입니다. 
   - **재료**: 버터, 밀가루, 설탕, 소금, 바닐라 익스트랙트
   - **사진**: 제공할 수 있는 사진이 없습니다.
   
2. **초코칩 쿠키**
   - 달콤한 초콜릿이 가득한 인기 있는 쿠키로, 아이들과 함께 만들기 좋습니다.
   - **재료**: 중력분, 버터, 설탕, 갈색설탕, 계란, 초콜릿칩
   - **사진**: 제공할 수 있는 사진이 없습니다.

3. **오트밀 쿠키**
   - 건강한 귀리의 맛과 함께 씹는 식감이 좋은 오트밀 쿠키입니다.
   - **재료**: 오트밀, 밀가루, 설탕, 버터, 계란
   - **사진**: 제공할 수 있는 사진이 없습니다.

원하는 쿠키를 말씀해 주시면, 그에 대한 자세한 레시피를 안내해 드리겠습니다!


In [42]:
history_id = mkhisid("user_id")
chain = mkch(version6())
while True:
    query = input("메시지 입력 > ")
    if query == "종료":
        break
    res = chain.invoke({"question": query}, config={"configurable": {"user_id": "user_id", "history_id": history_id}})
    print(res)

현재 저장된 히스토리: {('user_id', 'GKEC65'): InMemoryHistory(messages=[HumanMessage(content='쿠키 만드는 법 알려줘 나는 땅콩 알레르기가 있어', additional_kwargs={}, response_metadata={}), AIMessage(content='땅콩 알레르기가 있으시다면 땅콩을 사용하지 않는 쿠키 레시피를 추천해 드리겠습니다. 다음과 같은 쿠키를 고려해 보세요:\n\n1. **버터 쿠키**\n   - 고소한 맛과 바삭한 식감이 매력적인 기본적인 버터 쿠키입니다. \n   - **재료**: 버터, 밀가루, 설탕, 소금, 바닐라 익스트랙트\n   - **사진**: 제공할 수 있는 사진이 없습니다.\n   \n2. **초코칩 쿠키**\n   - 달콤한 초콜릿이 가득한 인기 있는 쿠키로, 아이들과 함께 만들기 좋습니다.\n   - **재료**: 중력분, 버터, 설탕, 갈색설탕, 계란, 초콜릿칩\n   - **사진**: 제공할 수 있는 사진이 없습니다.\n\n3. **오트밀 쿠키**\n   - 건강한 귀리의 맛과 함께 씹는 식감이 좋은 오트밀 쿠키입니다.\n   - **재료**: 오트밀, 밀가루, 설탕, 버터, 계란\n   - **사진**: 제공할 수 있는 사진이 없습니다.\n\n원하는 쿠키를 말씀해 주시면, 그에 대한 자세한 레시피를 안내해 드리겠습니다!', additional_kwargs={}, response_metadata={})]), ('user_id', '3JTHJQ'): InMemoryHistory(messages=[HumanMessage(content='쿠키 만드는 법 알려줘 나는 땅콩 알레르기가 있어', additional_kwargs={}, response_metadata={}), AIMessage(content='땅콩 알레르기가 있으시다면 땅콩이 들어가지 않은 쿠키 레시피를 추천해 드리겠습니다. 다음은 바나나가 안 들어간 바나나 쿠키의 레시피입니다.\n\n### 바나나가 안 